In [27]:
import pandas as pd
import numpy as np
from skimpy import clean_columns
import pycountry
import pycountry_convert as pc

In [28]:
# Read the CSV file
baseline = pd.read_csv("data_raw/baseline_raw_28_01_25.csv")
# Clean column names
baseline=clean_columns(baseline)
# remove useless columns
baseline = baseline.drop(columns=["identifier", "started"] + list(baseline.loc[:, "location_ts":"context_weather_timezone"].columns))
# remove useless rows
baseline = baseline.drop([0,1])

In [29]:
# Ensure the 'finished' column is in datetime format
baseline[['finished', 'registration_travel_travel_date']] = baseline[['finished', 'registration_travel_travel_date']].apply(pd.to_datetime, format='ISO8601')

In [30]:
# Define the complex transformation function
def transform_row(row):
    if not pd.isna(row['registration_health_chronics_0']):
        return "None"
    elif not pd.isna(row['registration_health_chronics_1']) and pd.isna(row['registration_health_chronics_2']) and pd.isna(row['registration_health_chronics_3']) and pd.isna(row['registration_health_chronics_4']):
        return "High blood pressure"
    elif pd.isna(row['registration_health_chronics_1']) and not pd.isna(row['registration_health_chronics_2']) and pd.isna(row['registration_health_chronics_3']) and pd.isna(row['registration_health_chronics_4']):
        return "Diabetes"
    elif pd.isna(row['registration_health_chronics_1']) and pd.isna(row['registration_health_chronics_2']) and not pd.isna(row['registration_health_chronics_3']) and pd.isna(row['registration_health_chronics_4']):
        return "Heart diseases"
    elif pd.isna(row['registration_health_chronics_1']) and pd.isna(row['registration_health_chronics_2']) and pd.isna(row['registration_health_chronics_3']) and not pd.isna(row['registration_health_chronics_4']):
        return "Immunosuppression"
    else:
        return "Multiple"

baseline['health_chronic'] = baseline.apply(transform_row, axis=1)

In [31]:
def transform_registration_basics_gender(x):
    if x == "0":
        return "Male"
    elif x == "1":
        return "Female"
    elif x == "2":
        return "Other"
    else:
        return np.nan
    
baseline['registration_basics_gender'] = baseline['registration_basics_gender'].apply(transform_registration_basics_gender)

In [32]:
def transform_registration_health_smoking(x):
    if x == "0":
        return "Not smoking"
    elif x == "1":
        return "Daily"
    elif x == "2":
        return "Weekly"
    elif x == "3":
        return "Monthly"
    elif x == "4":
        return "Former smoker"
    else:
        return np.nan

baseline['registration_health_smoking'] = baseline['registration_health_smoking'].apply(transform_registration_health_smoking)

In [33]:
# Filter test data
baseline = baseline[baseline['registration_travel_travel_purpose'] != '4: testing push']
baseline = baseline[baseline['registration_travel_travel_purpose'] != "4: test"]
baseline = baseline[baseline['registration_travel_travel_purpose'] != "5: test"]
baseline = baseline[baseline['registration_travel_travel_purpose'] != "4: teat 130"]

def transform_registration_travel_purpose(x):
    if x == "0":
        return "Leisure/tourist travellers"
    elif x == "1":
        return "Business/corporate travellers"
    elif x == "2":
        return "Visiting friends and relatives (VFR)"
    elif x == "3":
        return "Mass gathering events (Hajj, Olympics, World Cup)"
    elif x == "4: festival":
        return "Mass gathering events (Hajj, Olympics, World Cup)"
    elif x in ["4: Cross border worker (and also family)", "4: work", "4: in einer Schule arbeiten (freiwillig)","4: My Job (Flight Attendant)","4: Cabin Crew","4: volunteering ",
               "4: mission humanitaire", "4: conference ", "4: volunteer work in a school", "4: volunteerwork", "4: Volunteer ","4: volontariat","4: work in a hospital",
               "4: Dreharbeiten","4: voluntaria","4: Voyage humanitaire","4: construction","4: NGO project","4: Weltwärts Dienst","4: work with surgeons","4: tourism and voluntary work"]:
        return "Work"
    elif x in ["4: formation ", "4: intership", "4: Internship", "4: Research", "4: research exchange ","4: Studium","4: University elective","4: utlandsstudier",
               "4: stage en médecine", "4: stage médecine tropicale ", "4: stage en médecine", "4: studie",
               "4: stage médical ", "4: Uni (Austauschsemester?", "4: Auslandssemester", "4: mission humanitaire",
               "4: conference ", "4: Auslands Semester von der UZH", "4: forschungs Praktikum",
               "4: travail de Master en médecine", "4: Travail de terrain pour mon mémoire de master",
               "4: Austauschstudium","4: Study","4: Tourist und Ausbildung","4: congres","4: conference","4: attending lectures",
               "4: Voyage humanitaire "]:
        return "Education"
    elif x in ["4: Altro", "4: Other", "5: moldova", "4: Autre", "4: Andere", "4: Militärdienst", "4: Freiwilligendienst",
               "4: Auswandern"]:
        return "Other"
    elif x in ["4: Hochzeit unseres Sohnes in Indien","4: Besuch meiner Kinderpoliklink"]:
        return "Visiting friends and relatives (VFR)"
    elif x in ["4: Tourist travelling and visiting friends", "5: Besuch Eishockeyturnier der Schweizer U16 Nationalmannschaft ",
               "4: Weihnachten/Neujahr", "4: Visiting", "4: Aufenthalt in einem Ashram ", "4: tourism, NGO","4: holiday "]:
        return "Leisure/tourist travellers"
    elif x in ["5: Беженец / мигрант (пожалуйста, укажите страну происхождения)", "5: Біженець / мігрант (будь ласка, введіть країну походження)",
               "5: Ukraine", "5: Ukraine ", "5: Украина ", "5: Украина", "5: украина", "5: Швейцария ",
               "5: Refugee / Migrant (please enter country of origin)", "5: Flüchtling / Migrant (bitte Herkunftsland angeben)",
               "5: refugee/Ukraine ","5: Україна","5: Réfugié / Migrant (veuillez indiquer le pays d'origine)"]:
        return "Refugee/Ukraine"
    elif pd.isna(x):
        return np.nan
    else:
        return x
baseline['registration_travel_travel_purpose'] = baseline['registration_travel_travel_purpose'].apply(transform_registration_travel_purpose)

In [34]:
baseline['registration_travel_travel_purpose'].value_counts()

registration_travel_travel_purpose
Leisure/tourist travellers                           1075
Business/corporate travellers                         260
Visiting friends and relatives (VFR)                  243
Refugee/Ukraine                                        58
Education                                              28
Mass gathering events (Hajj, Olympics, World Cup)      25
Work                                                   20
Other                                                  17
Name: count, dtype: int64

In [35]:
# Rename the columns
baseline = baseline.rename(columns={
    'user_id': 'user_id',
    'id': 'trip_id',
    'finished': 'baseline_date',
    'registration_basics_age': 'age',
    'registration_basics_gender': 'gender',
    'registration_travel_country': 'country',
    'registration_travel_country_identifier': 'country_iso2c',
    'registration_travel_travel_purpose': 'travel_purpose',
    'registration_travel_travel_date': 'travel_date',
    'registration_travel_travel_duration': 'travel_duration',
    'registration_health_smoking': 'smoking_status'
})

In [36]:
# Function to convert ISO2 country code to country name
def iso2_to_country_name(iso2_code):
    try:
        return pycountry.countries.get(alpha_2=iso2_code).name
    except AttributeError:
        return None

# Convert country_iso2c to uppercase
baseline['country_iso2c'] = baseline['country_iso2c'].str.upper()

# Apply the function to create a new column country_clean
baseline['country_clean'] = baseline['country_iso2c'].apply(iso2_to_country_name)

In [37]:
# Function to convert ISO2 country code to continent name
def iso2_to_continent(iso2_code):
    try:
        continent_code = pc.country_alpha2_to_continent_code(iso2_code)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except KeyError:
        return None

# Apply the first condition for specific continent names
def determine_continent(iso2_code):
    if iso2_code in ["AFRICA", "AMERICAS", "ASIA"]:
        return iso2_code.title()
    elif iso2_code == "EU":
        return "Europe"
    else:
        continent = iso2_to_continent(iso2_code)
        if continent in ["South America", "North America", "Americas"]:
            return "Americas"
        else:
            return continent


# Apply the function to create a new column continent_clean
baseline['continent_clean'] = baseline['country_iso2c'].apply(determine_continent)

In [38]:
baseline = baseline[baseline.columns[:baseline.columns.get_loc('country_iso2c') + 1].tolist() + ['country_clean', 'continent_clean'] + baseline.columns[baseline.columns.get_loc('country_iso2c') + 1:].difference(['country_clean', 'continent_clean']).tolist()]

In [39]:
baseline = baseline.drop(columns=['country','registration_health_chronics_0','registration_health_chronics_1','registration_health_chronics_2','registration_health_chronics_3','registration_health_chronics_4'])

In [40]:
baseline[['age', 'travel_duration']] = baseline[['age', 'travel_duration']].astype('float64')

In [41]:
# check duplicate
baseline[baseline.duplicated(subset=['trip_id'], keep=False)]
# Filter the DataFrame to include rows with unique trip_id values
baseline = baseline.drop_duplicates(subset=['trip_id'])

In [42]:
# Identify the rows that match the specified conditions (devlopper test)
to_drop = baseline[
    (baseline['user_id'] == "71dWvTtxniTevm5Wrw7wazKjit93") & (baseline['trip_id'] == "SJBoI6Fef46aDKZvpyp3") |
    (baseline['user_id'] == "dPhQWSZVVFWrR1kWAGACyMLWXzW2") & (baseline['trip_id'] == "Zrgm5OEyv1YKdUTi2IRr") |
    (baseline['user_id'] == "hheBd0VfyLZLE43RaHbRPAidZmH2") & (baseline['trip_id'] == "n6Ec93wurCFoe4RyARDi") |
    (baseline['user_id'] == "TUWZNk1ALPWqEVPSRjeLb55gM7h1") |
    (baseline['user_id'] == "VhNJxOlWpyOAcQK7fdlaZdiZJg13") |
    (baseline['user_id'] == "tnouLlta6AMr0NtgQu1pTTNxy503")
]

# Drop the rows that match the specified conditions
filtered_baseline = baseline.drop(to_drop.index)

In [43]:
baseline['trip_number'] = baseline.sort_values(by=['user_id', 'travel_date']).groupby('user_id').cumcount() + 1
# Define a function to label trip_number automatically
def label_trip_number(n):
    return f"Trip {n}"

# Apply the function to trip_number
baseline['trip_number'] = baseline['trip_number'].apply(label_trip_number)

In [44]:
baseline.to_pickle('data_clean/baseline_clean.pkl')

In [45]:
baseline.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1726 entries, 2 to 1759
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   trip_id          1726 non-null   object        
 1   user_id          1726 non-null   object        
 2   baseline_date    1726 non-null   datetime64[ns]
 3   age              1726 non-null   float64       
 4   gender           1721 non-null   object        
 5   country_iso2c    1726 non-null   object        
 6   country_clean    1371 non-null   object        
 7   continent_clean  1719 non-null   object        
 8   health_chronic   1726 non-null   object        
 9   latitude         1484 non-null   float64       
 10  longitude        1484 non-null   float64       
 11  smoking_status   1725 non-null   object        
 12  travel_date      1726 non-null   datetime64[ns]
 13  travel_duration  1726 non-null   float64       
 14  travel_purpose   1726 non-null   object      